# <center> Machine Learning Based Approach to Predict Attacks on Industrial Control Systems </center>

## Problem Definition
Detecting /Predicting attacks on industrial systems using Machine Learning techniques.

## Project Outline 
- Apply machine learning techniques to predict attacks on industrial systems using HAI time series dataset.
- Evaluate and analyze dataset time series aspects ( stationary , seasonality ,etc ) and attributes. 
- Visualize the process using different visualization techniques. 

## HAI Security Dataset 
For this project we will investigate the The HIL-based augmented ICS security (HAI) time series database. The dataset is taken from HIL-based Augmented ICS research page on github.com.
Quoting the project site the HAI dataset was collected from a realistic industiral control system (ICS) testbed augmented with a Hardware-In-the-Loop (HIL) simulator that emulates steam-turbine power generation and pumped-storage hydropower generation.

According to the site also the HAI testbed is comprised the following physical control systems:
1. <b>a GE turbine</b>: Turbine Process (P2): 
2. <b>Emerson boiler</b>: Boiler Process (P1): 
3. <b>FESTO water treatment systems</b>: Water-treatment (P3) 
4. <b>All combined using a hardware-in-the-loop (HIL) simulator</b>: HIL (P4) HIL systems is composed of controller under a test a real time simulator together with IO Modules.
    
<br>
The dataset has multiple channels of measurements (e.g., sensors, actuators, control devices) represent the current status of the systems.

There are two major versions of this dataset. Each version consists of several csv files , and the site claims they satisfy time continuty.

Scenario configuration have been continuosuly developed  as follows:

* <b>Normal Operation</b>: Complex process of the long-term human intervention for normal operations applied.
<br>
<i>Situation</i>: <br>
Through the experiments , normal ranges are confirmed and monitored  even using a scheduled task to generate random values within the normal range.The normal ranges of SP values in which the entire process was stable were determined by experimentally changing the value of each SP.An HMI operation task scheduler was used to periodically set the SPs and HIL simulator.  variables to predefined values within the normal range.


* <b>Abnormal Behaviors</b>:  Realization of the various ICS attacks on real world systems using scalable attack tool followed by labeling of anomalies.

Three ICS are interconnected via HIL simulator that Simulated those complex systems.
<br>
<i>Situation</i>:<br>

Abnormal behavior occurred when some of the parameters were not within the limits of the normal range or were in unexpected states due to attacks, malfunctions, and failures
Attack scenarios have been implemented by considering attack target  for each feed back control loop. 

<i>Attack types</i>: 

-  a. Response Prevention: hiding abnormal response on PV on HMI 
-  b. SP attack: forcing the SP value to indirectly change the CO value 
-  c. CO attack: forcing the CO value directly 

The dataset attributes comprised from the followings:

1. <b>time</b> : This is the first column and it represents the observed local time as “yyyy-MM-dd hh:mm:ss,” while the rest columns provide the recorded SCADA data points - column 01 
2. <b>P1_B2004...P4_HT_LD </b>:  contains data collected from different setpoints in the archeticture
3. <b>attack</b>: provides an info for whether an attack occured or not. Where this is applicable to any attack happening on all the processes.last column.

## Model Proposal & Procedures

The diagram below illustrates our road map :


<img src="images/model-diagram.png" align="center" alt="Model Arch. Diagram"/>
My proposed ML evaluation  can be described as follows:


1. Loading the dataset and starting analyzing the data and the features.

2. Analayze the attributes and understand the process

3. Apply features selection and scaling 

4. Check for Imbalance in the dataset and balance if needed 

5. Time series data test - stationary trend etc 

6. Evaluate Machine Learning Model following this procudere:
  - Plot the data and identify unusual obeservations finding pattern in the data
  - Apply for transformation or differencing to remove trend and stabilize variance 
  - Test for stationary if not then apply another transforamtion or differencing 
  - Plot PACF and ACF to estimate the order of MA or AR process.
  - Try different combination of orders  and select the model with the lowest AIC 
  - Check for residuals - Ljung Box test 
  - Calculate predictions or forecats.




## Packages needed
Uncomment the following line and paste it in to a codebox:

all done via pip
* #!pip install yellowbrick
* #!pip install seaborn
* #!pip install scipy
* #!pip install statsmodels  


## Import Libraries 
Import some of the required libraries 


In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
from collections import Counter
from numpy import where
from imblearn.over_sampling import SMOTE
import datetime
datetime.timedelta()
from pandas import *
#Suppressing warnings
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error
from yellowbrick.classifier import ClassificationReport, ROCAUC, ClassBalance
from sklearn.decomposition import PCA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.api import VAR
from statsmodels.tools.eval_measures import rmse, aic
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.stattools import grangercausalitytests, adfuller
from tqdm import tqdm_notebook
from itertools import product
import statsmodels.api as sm
import time


In [73]:
# Seaborn Styling 
sns.set(
    font_scale=1.5,
    style="whitegrid",
    rc={'figure.figsize':(20,7)}
)

## Load the HAI Dataset 

First we start importing the dataset.

In [74]:
print("Reading the HAI data set===========================================")
hai_data = pd.read_csv('dataset/test2.csv.gz', compression='gzip',parse_dates=['time'], index_col=['time'])
hai_data.head()

Reading the HAI data set===========================================


,P1_B2004,P1_B2016,P1_B3004,P1_B3005,P1_B4002,P1_B4005,P1_B400B,P1_B4022,P1_FCV01D,P1_FCV01Z,...,P4_ST_GOV,P4_ST_LD,P4_ST_PO,P4_ST_PS,P4_ST_PT01,P4_ST_TT01,attack,attack_P1,attack_P2,attack_P3
time,,,,,,,,,,,,,,,,,,,,,
2020-07-09 15:00:00,0.09874,1.25036,382.21698,1118.61084,32.0,99.98731,2835.65454,35.37903,92.69160,92.51556,...,15844.0,283.92651,293.51129,0,10051.0,27600.0,0,0,0,0
2020-07-09 15:00:01,0.09874,1.24944,382.21698,1118.61084,32.0,100.00000,2832.98731,35.37674,92.64934,92.51556,...,15808.0,283.36591,292.67938,0,10052.0,27600.0,0,0,0,0
2020-07-09 15:00:02,0.09874,1.24746,382.21698,1118.61084,32.0,99.98115,2842.25244,35.37178,92.91075,92.51556,...,15734.0,282.93189,291.90179,0,10052.0,27595.0,0,0,0,0
2020-07-09 15:00:03,0.09874,1.24624,382.21698,1118.61084,32.0,100.00000,2833.12744,35.36873,92.92407,92.51556,...,15710.0,282.06378,291.59430,0,10053.0,27590.0,0,0,0,0
2020-07-09 15:00:04,0.09874,1.24364,382.21698,1118.61084,32.0,100.00000,2839.44482,35.36224,93.05405,92.51556,...,15608.0,283.67334,289.87628,0,10052.0,27588.0,0,0,0,0


In [76]:
df11 = pd.read_csv('dataset/test2.csv.gz', compression='gzip',header=0)
df11.time = pd.to_datetime(df11.time, format='%Y-%m-%d %H:%M:%S')
df11 = df11.sort_values(by="time")
hai_df11 = df11.reset_index(drop=True)
hai_df11.head(5)
hai_df11 = hai_df11.set_index('time')

In [77]:
hai_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 118801 entries, 2020-07-09 15:00:00 to 2020-07-11 00:00:00
Data columns (total 83 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   P1_B2004     118801 non-null  float64
 1   P1_B2016     118801 non-null  float64
 2   P1_B3004     118801 non-null  float64
 3   P1_B3005     118801 non-null  float64
 4   P1_B4002     118801 non-null  float64
 5   P1_B4005     118801 non-null  float64
 6   P1_B400B     118801 non-null  float64
 7   P1_B4022     118801 non-null  float64
 8   P1_FCV01D    118801 non-null  float64
 9   P1_FCV01Z    118801 non-null  float64
 10  P1_FCV02D    118801 non-null  float64
 11  P1_FCV02Z    118801 non-null  float64
 12  P1_FCV03D    118801 non-null  float64
 13  P1_FCV03Z    118801 non-null  float64
 14  P1_FT01      118801 non-null  float64
 15  P1_FT01Z     118801 non-null  float64
 16  P1_FT02      118801 non-null  float64
 17  P1_FT02Z     118801 non-null  flo

Looking at the dataset above , we can see there are different attributes with different scales on the data


## Data Cleaning 

So before we visualize the dataset attributes and apply our models we need to clean the dataset. For example , its important to remove features that can cause clutters which are features with single value.

In [78]:
# Let's look in to identifying all the null values from the attributes
hai_data.isna().sum()
hai_data=hai_data.dropna(axis=1)

In [79]:
print(hai_data.nunique())
# only contain one distinct value drop
keep = [c for c
        in list(hai_data)
        if len(hai_data[c].unique()) > 1]

hai_data = hai_data[keep]

P1_B2004         99
P1_B2016      18797
P1_B3004        119
P1_B3005        120
P1_B4002         96
              ...  
P4_ST_TT01      111
attack            2
attack_P1         2
attack_P2         2
attack_P3         2
Length: 83, dtype: int64


In [80]:
hai_data.shape

(118801, 64)

In [81]:
hai_data.columns

Index(['P1_B2004', 'P1_B2016', 'P1_B3004', 'P1_B3005', 'P1_B4002', 'P1_B4005',
       'P1_B400B', 'P1_B4022', 'P1_FCV01D', 'P1_FCV01Z', 'P1_FCV02D',
       'P1_FCV02Z', 'P1_FCV03D', 'P1_FCV03Z', 'P1_FT01', 'P1_FT01Z', 'P1_FT02',
       'P1_FT02Z', 'P1_FT03', 'P1_FT03Z', 'P1_LCV01D', 'P1_LCV01Z', 'P1_LIT01',
       'P1_PCV01D', 'P1_PCV01Z', 'P1_PCV02D', 'P1_PCV02Z', 'P1_PIT01',
       'P1_PIT02', 'P1_TIT01', 'P1_TIT02', 'P2_24Vdc', 'P2_CO_rpm', 'P2_Emerg',
       'P2_HILout', 'P2_OnOff', 'P2_SIT01', 'P2_SIT02', 'P2_VT01', 'P2_VXT02',
       'P2_VXT03', 'P2_VYT02', 'P2_VYT03', 'P3_FIT01', 'P3_LCP01D',
       'P3_LCV01D', 'P3_LIT01', 'P3_PIT01', 'P4_HT_FD', 'P4_HT_LD', 'P4_HT_PO',
       'P4_HT_PS', 'P4_LD', 'P4_ST_FD', 'P4_ST_GOV', 'P4_ST_LD', 'P4_ST_PO',
       'P4_ST_PS', 'P4_ST_PT01', 'P4_ST_TT01', 'attack', 'attack_P1',
       'attack_P2', 'attack_P3'],
      dtype='object')

## Understanding the Data
We will start by using Descriptive Statistics in Data Exploration. This is used to so we can  take a closer look at our loaded data.

### Descriprtive statistics
Let’s summarize the distribution of each attribute.

In [82]:
def test_stationarity(ts_data, column='', signif=0.05, series=False):
    if series:
        adf_test = adfuller(ts_data, autolag='AIC')
    else:
        adf_test = adfuller(ts_data[column], autolag='AIC')
    p_value = adf_test[1]
    if p_value <= signif:
        test_result = "Stationary"
    else:
        test_result = "Non-Stationary"
    return test_result

In [83]:
adf_test_results = { col: test_stationarity(hai_df11, col) for col in hai_df11.columns}
adf_test_results

{'P1_B2004': 'Non-Stationary',
 'P1_B2016': 'Stationary',
 'P1_B3004': 'Stationary',
 'P1_B3005': 'Non-Stationary',
 'P1_B4002': 'Non-Stationary',
 'P1_B4005': 'Stationary',
 'P1_B400B': 'Stationary',
 'P1_B4022': 'Stationary',
 'P1_FCV01D': 'Stationary',
 'P1_FCV01Z': 'Stationary',
 'P1_FCV02D': 'Stationary',
 'P1_FCV02Z': 'Stationary',
 'P1_FCV03D': 'Stationary',
 'P1_FCV03Z': 'Stationary',
 'P1_FT01': 'Stationary',
 'P1_FT01Z': 'Stationary',
 'P1_FT02': 'Stationary',
 'P1_FT02Z': 'Stationary',
 'P1_FT03': 'Stationary',
 'P1_FT03Z': 'Stationary',
 'P1_LCV01D': 'Stationary',
 'P1_LCV01Z': 'Stationary',
 'P1_LIT01': 'Stationary',
 'P1_PCV01D': 'Stationary',
 'P1_PCV01Z': 'Stationary',
 'P1_PCV02D': 'Stationary',
 'P1_PCV02Z': 'Stationary',
 'P1_PIT01': 'Stationary',
 'P1_PIT02': 'Stationary',
 'P1_PP01AD': 'Non-Stationary',
 'P1_PP01AR': 'Non-Stationary',
 'P1_PP01BD': 'Non-Stationary',
 'P1_PP01BR': 'Non-Stationary',
 'P1_PP02D': 'Non-Stationary',
 'P1_PP02R': 'Non-Stationary',
 'P1_S